In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from keras.optimizers import Adam
from keras.models import Sequential, Model
import json
from keras.preprocessing.image import ImageDataGenerator

from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    #print (ds.Modality)
    #print ("Patient Position")
    #print (ds.PatientPosition)
    #print ("Body part")
    
    if (ds.BodyPartExamined != "CHEST") and (ds.BodyPartExamined != "RIBCAGE"):
        print ("Wrong body part, so discontinuing")
        return
    
    if (int(ds.PatientAge) > 100): 
        print ("Age beyond model limits, so discontinuing")
        return
    
    return img

    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
    proc_img = (img - img_mean)/img_std
    proc_img =  resize(proc_img, img_size, anti_aliasing=True)
    return proc_img


# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json) 
    loaded_model.summary()
    
    loaded_model.load_weights(weight_path)
    print("Loaded model from disk")

    my_model = loaded_model
    optimizer = Adam(lr=1e-4)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    my_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return my_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_Y = model.predict(img, batch_size=1, verbose = True)  
    prediction=0
    if pred_Y > thresh:
        prediction=1
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
#test_dicoms = ['test1.dcm']

model_path = "my_model.json"
weight_path="{}_my_model.best.hdf5".format('xray_class')

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
#img_mean = # loads the mean image value they used during training preprocessing
#img_std = # loads the std dev image value they used during training preprocessing

# load json and create model

# load weights into new model
my_model = load_model (model_path, weight_path)

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    #mean_intensity = np.mean(img)
    #std_intensity = np.std(img)
    mean_intensity = 0
    std_intensity = 255
    img_proc = preprocess_image(img, mean_intensity, std_intensity, IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh=0.52)
    if (pred == 1):
        print("Suspected Case of Pneumonia")
    elif (pred == 0):
        print ("No Indication of Pneumonia")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)              